In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn
import re
import datetime
import dateutil.parser
import statistics 


In [2]:
df = pd.read_csv('Indian Earthquakes List Update_Magnitudes (1).csv',header=8, encoding = "ISO-8859-1", parse_dates=True)

/home/varun/LE-Course/Sem2/SMAI/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (0,1,2,3,6,7,9,10,11,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.columns = ( df.iloc[0].fillna('') + df.iloc[1].fillna(''))
df.columns = np.vectorize(lambda x: x.strip())(df.columns)

In [4]:
df = df.drop([0,1])
df = df.reset_index(drop=True)

In [5]:
df

,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME(UTC),(IST),MAGNITUDEMw,Mw,Mb,Mb,Ms,ML,LAT (N),LONG (E),DEPTH (km),INTENSITYMM,MMI,MME,LOCATION,REFERENCE
0,1,-2474,0,0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0,NaN,NaN,NaN,NaN,Dr STGR
1,2,-325,0,0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0,NaN,NaN,NaN,NaN,Dr STGR
2,3,25,0,0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,72.9,33.72,0,NaN,NaN,NaN,NaN,Dr STGR
3,4,26,5,10,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,5.698851894,5.999685205,17.3,80.1,NaN,NaN,NaN,NaN,NaN,NEIC
4,5,26,5,10,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,6.075520196,5.999685205,26,97,80,NaN,NaN,NaN,Myanmar,G-R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52984,52970,2019,7,28,19:24:08,00:54:08/2019-07-29/,3.2,3.2,NaN,3.07428,2.32377,2.91501,32.8°N,78.4°E,10,NaN,NaN,NaN,Jammu & Kashmir,NaN
52985,52971,2019,7,28,10:44:10,16:14:10,3.6,3.6,NaN,3.43659,2.78301,3.33473,25.5°N,90.4°E,70,NaN,NaN,NaN,"Distt. East Garo Hills ,Meghalaya",NaN
52986,52972,2019,7,28,21:25:06,7/29/2019 2:55,4,4,NaN,3.79891,3.24225,3.75446,23.2°N,86.5°E,22,NaN,NaN,NaN,"Distt. Puruliya, West Bengal",NaN
52987,52973,2019,7,29,3:33:53,9:03:53,4.3,4.3,NaN,4.07065,3.58668,4.06925,32.8°N,76.4°E,20,NaN,NaN,NaN,"Distt. Lahaul-Spiti,Himachal Pradesh",NaN


In [6]:
def ints(x):
    if pd.notna(x):
        if (type(x) == str and x.strip().isdigit()):
            return int(x.strip())
        elif (type(x) in [int, float]):
            return int(x)
        else:
            return -1
    return -1

In [7]:
def floats(x):
    if pd.notna(x):
        if (type(x) == str and x.strip().isdigit()):
            return int(x.strip())
        elif (type(x) in [int, float]):
            return float(x)
        else:
            return -1
    return -1

In [8]:
for col in ['YEAR','DATE','MONTH']:
#     df[col] = df[col].dropna()
    df[col] = df[col].apply(ints)

In [9]:
for col in ['Mw','MAGNITUDEMw']:
#     df[col] = df[col].dropna()
    df[col] = df[col].apply(floats)

In [10]:
df = df.drop(np.where((df['YEAR']<=0)|(df['MONTH']<=0) | (df['DATE']<=0))[0])

In [11]:
df = df.drop(df.index[df['Mw']<=0])

In [12]:
np.where(df['ORIGIN TIME(UTC)'].isna() & df['(IST)'].notna())

(array([], dtype=int64),)

In [13]:
np.where(df['ORIGIN TIME(UTC)'].notna() & df['(IST)'].isna())

(array([   15,    18,    22, ..., 19731, 19732, 19733]),)

In [14]:
np.where(df['MAGNITUDEMw'].notna() & df['Mw'].isna())

(array([], dtype=int64),)

In [15]:
np.where(df['MAGNITUDEMw'].isna() & df['Mw'].notna())

(array([], dtype=int64),)

In [16]:
def formatint(x):
    if pd.notna(x):
        if (type(x) == str and x.strip().isdigit()):
            return int(x.strip())
        elif (type(x) in [int, float]):
            return int(x)
        else:
#             print(x)
            return pd.NA
    return x

In [17]:
def formatfloat(x):
    if pd.notna(x):
        if (type(x) == str and x.strip().isdigit()):
            return int(x.strip())
        elif (type(x) in [int, float]):
            return float(x)
        else:
#             print(x)
            return pd.NA
    return x

In [18]:
df['MAGNITUDEMw'] = df['MAGNITUDEMw'].apply(formatfloat)

In [19]:
df

,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME(UTC),(IST),MAGNITUDEMw,Mw,Mb,Mb,Ms,ML,LAT (N),LONG (E),DEPTH (km),INTENSITYMM,MMI,MME,LOCATION,REFERENCE
332,318,1852,3,31,NaN,NaN,6.0,6.0,5.610507246,5.610507246,5.538461538,5.853095488,26.17,85.9,NaN,NaN,NaN,NaN,NaN,NEIC
449,435,1863,11,18,NaN,NaN,5.0,5.0,4.704710145,4.704710145,4.390355913,4.803777545,26.47,84.43,NaN,NaN,NaN,NaN,NaN,NEIC
572,558,1871,1,31,NaN,NaN,5.0,5.0,4.704710145,4.704710145,4.390355913,4.803777545,16.98,82.33,NaN,NaN,NaN,NaN,NaN,NEIC
590,576,1872,4,14,NaN,NaN,5.0,5.0,4.704710145,4.704710145,4.390355913,4.803777545,26.3,89.53,NaN,NaN,NaN,NaN,NaN,NEIC
645,631,1882,6,28,NaN,NaN,5.0,5.0,4.704710145,4.704710145,4.390355913,4.803777545,22.6,93.4,NaN,NaN,NaN,NaN,NaN,NEIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52984,52970,2019,7,28,19:24:08,00:54:08/2019-07-29/,3.2,3.2,NaN,3.07428,2.32377,2.91501,32.8°N,78.4°E,10,NaN,NaN,NaN,Jammu & Kashmir,NaN
52985,52971,2019,7,28,10:44:10,16:14:10,3.6,3.6,NaN,3.43659,2.78301,3.33473,25.5°N,90.4°E,70,NaN,NaN,NaN,"Distt. East Garo Hills ,Meghalaya",NaN
52986,52972,2019,7,28,21:25:06,7/29/2019 2:55,4.0,4.0,NaN,3.79891,3.24225,3.75446,23.2°N,86.5°E,22,NaN,NaN,NaN,"Distt. Puruliya, West Bengal",NaN
52987,52973,2019,7,29,3:33:53,9:03:53,4.3,4.3,NaN,4.07065,3.58668,4.06925,32.8°N,76.4°E,20,NaN,NaN,NaN,"Distt. Lahaul-Spiti,Himachal Pradesh",NaN


In [20]:
df = df[['YEAR', 'MONTH', 'DATE', 'ORIGIN TIME(UTC)', 'Mw', 'LAT (N)', 'LONG (E)',
       'DEPTH (km)', 'REFERENCE']]

In [21]:
df.head()

,YEAR,MONTH,DATE,ORIGIN TIME(UTC),Mw,LAT (N),LONG (E),DEPTH (km),REFERENCE
332,1852,3,31,NaN,6.0,26.17,85.9,NaN,NEIC
449,1863,11,18,NaN,5.0,26.47,84.43,NaN,NEIC
572,1871,1,31,NaN,5.0,16.98,82.33,NaN,NEIC
590,1872,4,14,NaN,5.0,26.3,89.53,NaN,NEIC
645,1882,6,28,NaN,5.0,22.6,93.4,NaN,NEIC


In [22]:
df['ORIGIN TIME(UTC)'].fillna('00:00:00',inplace=True)
df['DEPTH (km)'].fillna(0,inplace=True)
df['REFERENCE'].fillna('',inplace=True)

/home/varun/LE-Course/Sem2/SMAI/env/lib/python3.8/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [23]:
df['ORIGIN TIME(UTC)']!=0

332      True
449      True
572      True
590      True
645      True
         ... 
52984    True
52985    True
52986    True
52987    True
52988    True
Name: ORIGIN TIME(UTC), Length: 20160, dtype: bool

In [24]:
def time2str(x):
    if type(x) in [int,float]:
        x = x/1000.0
        return datetime.datetime.fromtimestamp(s).strftime('%H:%M:%S.%f')
    x = x.replace(' ',':')
    x = x.replace('.',':')
    flag = (x.find('PM') > -1)
    x = re.sub('[^0-9.:]*', '', x)
    
    x = np.array([ int(t) if t.isdigit() else -1 for t in x.split(':')])
    x = x[x>=0]
    
    x_ = np.zeros(4)
    try:
        x_[:x.shape[0]] = x
    except:
        print(x)
    
    if flag:
        x_[0] = x_[0] + (12 if x_[0]<12 else 0)
    x_ = np.min(np.vstack([x_,np.array([23,59,59,99])]),axis = 0)
    return '{}:{}:{}.{}'.format(int(x_[0]),int(x_[1]),int(x_[2]),int(x_[3]))
#     return np.sum(x_ * np.array([3600,60,1,0.01]))

In [25]:
def cols2timestamp(row):
    year=row['YEAR']
    month=row['MONTH']
    day=row['DATE']
    time = '' if pd.isnull(row['ORIGIN TIME(UTC)']) else time2str(row['ORIGIN TIME(UTC)'])
    try:
        x = dateutil.parser.parse('{}-{}-{} {}'.format(year,month,day,time))
    except:
        x = dateutil.parser.parse('{}-{}-{} {}'.format(year,month,day-3,time))
    return x
    

In [26]:
df['TIMESTAMP'] = df.apply(cols2timestamp,axis = 1)

<ipython-input-26-052cde2ff581>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TIMESTAMP'] = df.apply(cols2timestamp,axis = 1)


In [27]:
df

,YEAR,MONTH,DATE,ORIGIN TIME(UTC),Mw,LAT (N),LONG (E),DEPTH (km),REFERENCE,TIMESTAMP
332,1852,3,31,00:00:00,6.0,26.17,85.9,0,NEIC,1852-03-31 00:00:00
449,1863,11,18,00:00:00,5.0,26.47,84.43,0,NEIC,1863-11-18 00:00:00
572,1871,1,31,00:00:00,5.0,16.98,82.33,0,NEIC,1871-01-31 00:00:00
590,1872,4,14,00:00:00,5.0,26.3,89.53,0,NEIC,1872-04-14 00:00:00
645,1882,6,28,00:00:00,5.0,22.6,93.4,0,NEIC,1882-06-28 00:00:00
...,...,...,...,...,...,...,...,...,...,...
52984,2019,7,28,19:24:08,3.2,32.8°N,78.4°E,10,,2019-07-28 19:24:08
52985,2019,7,28,10:44:10,3.6,25.5°N,90.4°E,70,,2019-07-28 10:44:10
52986,2019,7,28,21:25:06,4.0,23.2°N,86.5°E,22,,2019-07-28 21:25:06
52987,2019,7,29,3:33:53,4.3,32.8°N,76.4°E,20,,2019-07-29 03:33:53


In [28]:
def NS(x):
    if type(x) in [int,float]:
        return x
    
    flag = x.find('S')>-1
    x = re.sub('[^0-9.]', '', x)
    return float(x) * (-1 if flag else 1)

In [29]:
def EW(x):
    if type(x) in [int,float]:
        return x
    
    flag = x.find('W')>-1
    x = re.sub('[^0-9.]', '', x)
    return float(x) * (-1 if flag else 1)

In [30]:
df['LAT (N)'] = df['LAT (N)'].apply(NS)
df['LONG (E)'] = df['LONG (E)'].apply(EW)

<ipython-input-30-139af6abc6bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LAT (N)'] = df['LAT (N)'].apply(NS)
<ipython-input-30-139af6abc6bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LONG (E)'] = df['LONG (E)'].apply(EW)


In [31]:
df['label'] = df['Mw'] > 4.53

<ipython-input-31-b6c1dbef6b47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['Mw'] > 4.53


In [32]:
df

,YEAR,MONTH,DATE,ORIGIN TIME(UTC),Mw,LAT (N),LONG (E),DEPTH (km),REFERENCE,TIMESTAMP,label
332,1852,3,31,00:00:00,6.0,26.17,85.90,0,NEIC,1852-03-31 00:00:00,True
449,1863,11,18,00:00:00,5.0,26.47,84.43,0,NEIC,1863-11-18 00:00:00,True
572,1871,1,31,00:00:00,5.0,16.98,82.33,0,NEIC,1871-01-31 00:00:00,True
590,1872,4,14,00:00:00,5.0,26.30,89.53,0,NEIC,1872-04-14 00:00:00,True
645,1882,6,28,00:00:00,5.0,22.60,93.40,0,NEIC,1882-06-28 00:00:00,True
...,...,...,...,...,...,...,...,...,...,...,...
52984,2019,7,28,19:24:08,3.2,32.80,78.40,10,,2019-07-28 19:24:08,False
52985,2019,7,28,10:44:10,3.6,25.50,90.40,70,,2019-07-28 10:44:10,False
52986,2019,7,28,21:25:06,4.0,23.20,86.50,22,,2019-07-28 21:25:06,False
52987,2019,7,29,3:33:53,4.3,32.80,76.40,20,,2019-07-29 03:33:53,False


In [33]:
def total_in1deg_count(x):
    return df[(df['ORIGIN TIME(UTC)'] < x['ORIGIN TIME(UTC)']) & ((df['LAT (N)']>=x['LAT (N)'] -1) & (df['LAT (N)']<=x['LAT (N)'] + 1)) & ((df['LONG (E)']>=x['LONG (E)'] -1) & (df['LONG (E)']<=x['LONG (E)'] + 1))].shape[0]

In [37]:
# Add target column
# Add Last Earthquake at (X±0.5,Y±0.5)
# Add Number of Eathquake Before Last date
# Add Median Frequency at (X±0.5,Y±0.5)


latRange=0.5
longRange=0.5
lastEq=[]
medianFrequency=[]
numEq=[]
timeleft_ratio = []
for index,row in df.iterrows():
#     clear_output(wait=True)
#     print(index,row)
    lat=row['LAT (N)']
    long=row['LONG (E)']
    timeList=df['TIMESTAMP'][((df['LAT (N)']>=lat-latRange)&(df['LAT (N)']<=lat+latRange) )&((df['LONG (E)']<=long+longRange)&(df['LONG (E)']>=long-longRange))]
    lastEq.append(np.max(timeList[timeList<row['TIMESTAMP']].max(),-np.inf))
    numEq.append(timeList.shape[0])

    timeList=timeList.tolist()
    timeList.sort()


    mf_append = np.inf
    if len(timeList)>1:
        deqt=[] # Difference 
        for i in range(1,len(timeList)):
            deqt.append(timeList[i]-timeList[i-1])
        mf_append = statistics.median(deqt)
    medianFrequency.append(mf_append)
    
    timeleft_ratio.append((row['TIMESTAMP'] - lastEq[-1])/np.max(mf_append,0.0001))

df['Last Eq'] = lastEq
df['Frequency'] = medianFrequency
# df['Frequency(Years)'] = df['Frequency(Years)'].Year
df['numEq']=numEq
    



TypeError: Cannot compare type Timedelta with type float

In [38]:
lastEq[-1]

NaT

In [ ]:
df

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:
max([dateutil.parser.parse('0:0'),dateutil.parser.parse('1:0')])